# Single Fuel Train-Test Split | FEATURE ENGINEERING!!!


**Difference from v1 is that the coal sample is totally excluded.**

In this experiment each fuel haas been included as test set and excluded from training set. Each trained model was tried on selected models. Model performances within each trial has been saved as csv file at the end.

File loc: "C:\Users\demir\OneDrive\Desktop\MSc Thesis\Data\!Exp_data\experiment_results.csv"

In [92]:
import pandas as pd
import numpy as np
from sklearn.impute import KNNImputer
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.dummy import DummyRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
import xgboost as xgb
import lightgbm as lgb
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.svm import SVR
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error

# Load data
data_path = r"C:\Users\demir\OneDrive\Desktop\MSc Thesis\!Data_Ea\mad_t1_Ea.csv"
df = pd.read_csv(data_path, delimiter=',')
df.columns

Index(['Unnamed: 0', 'sample', 'wc', 'vm', 'fc', 'ac', 'c', 'h', 'o', 'n', 's',
       'cl', 'hc', 'oc', 'lhv', 'temperature', 'residence_time', 'pressure',
       'heat_rate', 'fuel_type', 'fuel_category', 'devol_yield', 'dalpha', 'y',
       'T_kelvin', 'ln_ln_1_over_y', 'ln_T', 'inv_T', 'E3_Broido_kJmol'],
      dtype='object')

In [99]:

# --- Initialize new columns ---
df['second_fuel_category'] = np.nan
df['third_fuel_category'] = np.nan

# --- Rule 1: Lignocellulosic ---
ligno_keywords = ['Hemicellulose', 'Lignin', 'Torr-Wood', 'Wood', 'Cellulose']
mask_ligno = df['fuel_type'].str.contains('|'.join(ligno_keywords), case=False, na=False)
df.loc[mask_ligno, 'second_fuel_category'] = 'Lignocellulosic'
df.loc[mask_ligno, 'third_fuel_category'] = 'Lignocellulosic'

# --- Rule 2: Anaerobic ---
anaerobic_keywords = ['Digestate', 'Sewage', 'HTC-MSW']
mask_anaerobic = df['fuel_type'].str.contains('|'.join(anaerobic_keywords), case=False, na=False)
df.loc[mask_anaerobic, 'second_fuel_category'] = 'Anaerobic'
df.loc[mask_anaerobic, 'third_fuel_category'] = 'Anaerobic'

# --- Rule 3: Plastic ---
mask_plastic = df['fuel_category'] == 'Plastic'
df.loc[mask_plastic, 'second_fuel_category'] = 'Plastic'
df.loc[mask_plastic, 'third_fuel_category'] = 'Plastic'

# --- Rule 4: Coal ---
mask_coal = df['fuel_category'] == 'Coal'
df.loc[mask_coal, 'second_fuel_category'] = 'Coal'
df.loc[mask_coal, 'third_fuel_category'] = 'Coal'

# --- Rule 5: Mix → both Plastic and Anaerobic ---
mask_mix = df['fuel_category'] == 'Mix'
df.loc[mask_mix, 'second_fuel_category'] = 'Plastic'
df.loc[mask_mix, 'third_fuel_category'] = 'Anaerobic'

# --- Apply One-Hot Encoding to both new columns ---
second_ohe = pd.get_dummies(df['second_fuel_category'], prefix='second_fc')
third_ohe  = pd.get_dummies(df['third_fuel_category'],  prefix='third_fc')

# --- Combine OHE columns with original DataFrame ---
df = pd.concat([df, second_ohe, third_ohe], axis=1)


C:\Users\demir\AppData\Local\Temp\ipykernel_23748\2044056379.py:9: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'Lignocellulosic' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.loc[mask_ligno, 'second_fuel_category'] = 'Lignocellulosic'
C:\Users\demir\AppData\Local\Temp\ipykernel_23748\2044056379.py:10: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'Lignocellulosic' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.loc[mask_ligno, 'third_fuel_category'] = 'Lignocellulosic'
C:\Users\demir\AppData\Local\Temp\ipykernel_23748\2044056379.py:11: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'Lignocellulosic' has dtype incompatible with float64, please explicit

,Unnamed: 0,wc,c,h,o,n,s,cl,hc,oc,...,T_kelvin,ln_ln_1_over_y,ln_T,inv_T,E3_Broido_kJmol,vm_fc_ratio,ac_fc_ratio,t_to_T,ho,T_residence
count,1149.000000,1149.000000,1149.000000,1149.000000,1149.000000,1149.000000,1149.000000,1149.000000,1149.000000,1149.000000,...,1149.000000,1149.000000,1149.000000,1149.000000,1149.000000,1149.000000,1149.000000,1149.000000,1149.000000,1149.000000
mean,1113.499565,6.733830,49.932586,5.786183,20.540763,1.719016,0.528549,0.375132,1.405672,0.347951,...,1070.208312,-0.368716,6.955575,0.000975,23.141168,5.589855,1.659163,4.277128,0.358325,4682.158399
std,646.165137,2.363019,12.330217,1.889256,10.088599,1.033650,0.673482,0.676754,0.395418,0.179236,...,203.322269,0.809057,0.206866,0.000228,9.647195,4.601771,1.889472,15.927434,0.247309,3884.464305
min,2.000000,2.600000,32.400000,3.265000,7.000000,0.269662,0.046420,0.000000,0.700000,0.000000,...,473.150000,-4.250424,6.159412,0.000679,8.855729,1.037978,0.000000,0.200000,0.082348,100.000000
25%,547.000000,5.200000,37.500000,4.600000,15.688000,0.900000,0.150000,0.000000,1.200000,0.200000,...,873.150000,-0.646707,6.772107,0.000785,17.085785,3.292208,0.102689,0.600000,0.196687,1000.000000
50%,1110.000000,7.345000,50.400000,4.910000,17.110000,1.832640,0.200000,0.000000,1.500000,0.326926,...,1073.150000,-0.225068,6.978354,0.000932,22.300701,4.597561,1.538961,0.800000,0.268849,4000.000000
75%,1681.000000,7.752000,58.160000,7.000000,21.200000,2.210000,0.500000,0.100000,1.675000,0.400000,...,1273.150000,0.090182,7.149249,0.001145,30.140345,8.377534,1.538961,1.000000,0.452594,8000.000000
max,2237.000000,10.800000,82.300000,10.132050,54.936839,3.900000,2.500000,1.800000,2.100000,1.044020,...,1473.150000,1.191419,7.295158,0.002113,44.128487,29.936651,5.723868,100.000000,1.000000,16000.000000


In [94]:
fuel_types = df['fuel_type'].unique()

# Optional: define a margin (e.g. 10%)
margin_ratio = 0.1
epsilon = 1e-6

# Create new ratio features
df['vm_fc_ratio'] = df['vm'] / (df['fc'] + epsilon)
df['ac_fc_ratio'] = df['ac'] / (df['fc'] + epsilon)
df['t_to_T'] = df['temperature'] / df['heat_rate']
df['ho'] = df['h'] / df['o']
df['T_residence'] = df['temperature'] * df['residence_time']
min_vm_fc_ratio = df['vm_fc_ratio'].min()
max_ac_fc_ratio = df['ac_fc_ratio'].max()
min_ac_fc_ratio = df['ac_fc_ratio'].min()
max_vm_fc_ratio = df['vm_fc_ratio'].max()
min_t_to_T = df['t_to_T'].min()
max_t_to_T = df['t_to_T'].max()
min_T_residence = df['T_residence'].min()
max_T_residence = df['T_residence'].max()
min_ho = df['ho'].min()
max_ho = df['ho'].max()
min_temperature = df['temperature'].min()
max_temperature = df['temperature'].max()
min_E3 = df['E3_Broido_kJmol'].min()
max_E3 = df['E3_Broido_kJmol'].max()


# Drop original columns
df.drop(columns=['vm', 'fc', 'ac'], inplace=True)

# Update feature list
feature_cols = [
    'h', 'wc', 'c', 'lhv', 'o', 'n', 's', 'cl',
    'hc', 'oc', 'temperature', 'residence_time', 'pressure', 'heat_rate',
    'vm_fc_ratio', 'ac_fc_ratio', 't_to_T', 'T_residence', 'ho', 'E3_Broido_kJmol'
]

# Also update min-max bounds accordingly (you may compute new ranges empirically or just fit on full data)
# These are your original min and max values (as from your summary)
min_values = {
    'wc': 2.6, 'vm': 44.2, 'fc': 3.064326, 'ac': 0.0, 'c': 32.4, 'h': 3.265 , 'lhv': 13.528,
    'o': 7.0, 'n': 0.269662, 's': 0.04642, 'cl': 0.0, 'hc': 0.7, 'oc': 0.0,
    'temperature': min_temperature, 'residence_time': 0.5, 'pressure': 0.5, 'heat_rate': 10, 'vm_fc_ratio': min_vm_fc_ratio, 'ac_fc_ratio': min_ac_fc_ratio, 't_to_T': min_t_to_T, 'T_residence': min_T_residence, 'ho': min_ho, 'E3_Broido_kJmol':min_E3
}
max_values = {
    'wc': 10.8, 'vm': 91.735674, 'fc': 44.5, 'ac': 37.52, 'c': 82.3, 'h': 10.13205, 'lhv': 35.8,
    'o': 54.936839, 'n': 3.9, 's': 2.5, 'cl': 1.5, 'hc': 1.857, 'oc': 1.04402, 'temperature': max_temperature, 'residence_time': 20, 'pressure': 20, 'heat_rate': 1000, 'vm_fc_ratio':max_vm_fc_ratio, 'ac_fc_ratio': max_ac_fc_ratio, 't_to_T': max_t_to_T, 'T_residence': max_T_residence, 'ho': max_ho, 'E3_Broido_kJmol':max_E3}


# Create expanded bounds
expanded_bounds = {}
for feat in feature_cols:
    min_val = min_values[feat]
    max_val = max_values[feat]
    range_val = max_val - min_val
    new_min = min_val - range_val * margin_ratio
    if new_min < 0:
        new_min = 0
    new_max = max_val + range_val * margin_ratio
    expanded_bounds[feat] = (round(new_min, 3), round(new_max, 3))

# Prepare fake data to fit the scaler
X_bounds = pd.DataFrame({col: [expanded_bounds[col][0], expanded_bounds[col][1]] for col in feature_cols})

# Fit a global scaler based on the expanded bounds
scaler = MinMaxScaler()
scaler.fit(X_bounds)

# Store results
results = []
print(pd.unique(df['residence_time']))
# Iterate over each fuel type
for test_fuel in fuel_types:
    print(f"Processing fuel type: {test_fuel}")
    train_data = df[df['fuel_type'] != test_fuel].drop(columns=['fuel_type','fuel_category', 'ln_T', 'inv_T', 'dalpha', 'y',
       'T_kelvin', 'ln_ln_1_over_y','Unnamed: 0', 'T_residence','second_fuel_category', 'third_fuel_category','h', 'wc', 'c', 'lhv', 'o', 'n', 's', 'cl','t_to_T' ]).reset_index(drop=True)
    test_data = df[df['fuel_type'] == test_fuel].drop(columns=['fuel_type','fuel_category', 'ln_T', 'inv_T', 'dalpha', 'y',
       'T_kelvin', 'ln_ln_1_over_y', 'Unnamed: 0', 'T_residence','second_fuel_category', 'third_fuel_category','h', 'wc', 'c', 'lhv', 'o', 'n', 's', 'cl','t_to_T' ]).reset_index(drop=True)
    X_train = train_data.drop(columns=['sample', 'devol_yield'])
    y_train = train_data['devol_yield']
    X_test = test_data.drop(columns=['sample', 'devol_yield'])
    y_test = test_data['devol_yield']
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    X_train_scaled = pd.DataFrame(X_train_scaled, columns=X_train.columns)
    X_test_scaled = pd.DataFrame(X_test_scaled, columns=X_test.columns)

    # Models
    models = {
        "KNN": KNeighborsRegressor(n_neighbors=5),
        "Linear": LinearRegression(),
        "Ridge": Ridge(alpha=1.0),
        "Lasso": Lasso(alpha=0.1),
        "ElasticNet": ElasticNet(alpha=0.1, l1_ratio=0.5),
        "DecisionTree": DecisionTreeRegressor(max_depth=5),
        "RandomForest": RandomForestRegressor(n_estimators=2000, max_depth=5),
        "GradientBoosting": GradientBoostingRegressor(n_estimators=2000, learning_rate=0.1),
        "XGBoost": xgb.XGBRegressor(n_estimators=2000, learning_rate=0.1, loss="Huber"),
        "LightGBM": lgb.LGBMRegressor(n_estimators=2000, learning_rate=0.1, loss="Huber"),
        "SVR": SVR(kernel='rbf', C=1.0, epsilon=0.1),
        "MLP": MLPRegressor(hidden_layer_sizes=(100,), activation='relu', max_iter=5000)
    }

    # Train and evaluate models
    for name, model in models.items():
        model.fit(X_train_scaled, y_train)
        # Predict on test set
        y_pred = model.predict(X_test_scaled)

        # Compute metrics
        r2   = model.score(X_test_scaled, y_test)
        mae  = mean_absolute_error(y_test, y_pred)
        rmse = mean_squared_error(y_test, y_pred)  # squared=False gives RMSE directly

        # Record results
        results.append({
            "Fuel Type": test_fuel,
            "Model":      name,
            "R2":         r2,
            "MAE":        mae,
            "RMSE":       rmse
        })

# After the loop, save as before
results_df = pd.DataFrame(results)
results_df.to_csv(
    r"C:\Users\demir\OneDrive\Desktop\MSc Thesis\!Data_Ea\all2_ohe_kinetic_X_huber_baselines.csv",    index=False
)

print("Experiment completed! Results (including MAE and RMSE) saved!")

[ 1.   0.5 10.   2.   5.   1.5 20. ]
Processing fuel type: Cellulose


C:\Users\demir\PycharmProjects\volatile-predict\.venv\lib\site-packages\xgboost\core.py:158: UserWarning: [22:14:32] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-08cbc0333d8d4aae1-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "loss" } are not used.

  warnings.warn(smsg, UserWarning)
C:\Users\demir\PycharmProjects\volatile-predict\.venv\lib\site-packages\lightgbm\sklearn.py:821: UserWarning: Found 'loss' in params. Will use it instead of 'objective' argument
  _log_warning(f"Found '{alias}' in params. Will use it instead of 'objective' argument")


[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000213 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 348
[LightGBM] [Info] Number of data points in the train set: 1128, number of used features: 18
[LightGBM] [Info] Start training from score 52.871465
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: 

C:\Users\demir\PycharmProjects\volatile-predict\.venv\lib\site-packages\lightgbm\sklearn.py:821: UserWarning: Found 'loss' in params. Will use it instead of 'objective' argument
  _log_warning(f"Found '{alias}' in params. Will use it instead of 'objective' argument")
C:\Users\demir\PycharmProjects\volatile-predict\.venv\lib\site-packages\lightgbm\sklearn.py:821: UserWarning: Found 'loss' in params. Will use it instead of 'objective' argument
  _log_warning(f"Found '{alias}' in params. Will use it instead of 'objective' argument")


Processing fuel type: HTC-MSW


...

Processing fuel type: Wood


C:\Users\demir\PycharmProjects\volatile-predict\.venv\lib\site-packages\xgboost\core.py:158: UserWarning: [22:17:27] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-08cbc0333d8d4aae1-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "loss" } are not used.

  warnings.warn(smsg, UserWarning)
C:\Users\demir\PycharmProjects\volatile-predict\.venv\lib\site-packages\lightgbm\sklearn.py:821: UserWarning: Found 'loss' in params. Will use it instead of 'objective' argument
  _log_warning(f"Found '{alias}' in params. Will use it instead of 'objective' argument")


[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000107 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 347
[LightGBM] [Info] Number of data points in the train set: 1029, number of used features: 18
[LightGBM] [Info] Start training from score 50.820794
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: 

C:\Users\demir\PycharmProjects\volatile-predict\.venv\lib\site-packages\lightgbm\sklearn.py:821: UserWarning: Found 'loss' in params. Will use it instead of 'objective' argument
  _log_warning(f"Found '{alias}' in params. Will use it instead of 'objective' argument")
C:\Users\demir\PycharmProjects\volatile-predict\.venv\lib\site-packages\lightgbm\sklearn.py:821: UserWarning: Found 'loss' in params. Will use it instead of 'objective' argument
  _log_warning(f"Found '{alias}' in params. Will use it instead of 'objective' argument")


Experiment completed! Results (including MAE and RMSE) saved!
